# Solar Panels
This notebook will be used to clean the solar panel data.

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

print('Pandas version:', pd.__version__) # 2.2.3
print('Numpy version:', np.__version__) # 1.26.4

Pandas version: 2.2.3
Numpy version: 1.26.4


# Load the dataset
The dataset was downloaded from <a href='https://opendata.cbs.nl/#/CBS/nl/dataset/85005NED/table'>cbs</a>. It contains the amount of solar panels in the Netherlands at the end of the years 2021, 2022 and 2023.

In [3]:
df = pd.read_csv('../data/Other/Zonnepanelen_2021-2024.csv', sep=';', skiprows=3)
df

,Regio's,Onderwerp,Unnamed: 2,2021,2022,2023 **,2024 eerste helft*
0,Nederland,Installaties,aantal,1730285,2298859,2875487,3023653
1,Nederland,Opgesteld vermogen van zonnepanelen,kWp,14822880,19536096,24301725,26065113
2,Nederland,Opgesteld vermogen omvormers,kW,NaN,17451365,21394768,22814464
3,Nederland,Productie van zonnestroom,mln kWh,11304,16659,19580,10984
4,Noord-Nederland (LD),Installaties,aantal,246365,309909,379524,398051
...,...,...,...,...,...,...,...
1516,Gemeenten; niet in te delen,Installaties,aantal,426,509,520,533
1517,Gemeenten; niet in te delen,Opgesteld vermogen van zonnepanelen,kWp,14389,17208,28679,175854
1518,Gemeenten; niet in te delen,Opgesteld vermogen omvormers,kW,NaN,14768,23689,140124
1519,Gemeenten; niet in te delen,Productie van zonnestroom,mln kWh,12,16,25,28


# Data Cleaning
First we will drop the last row, since this does not contain any data but just states the source.

In [4]:
if df["Regio's"].iloc[-1] == 'Bron: CBS':
    df = df.drop(index=df.index[-1])

df.tail(5)

,Regio's,Onderwerp,Unnamed: 2,2021,2022,2023 **,2024 eerste helft*
1515,Zwolle,Productie van zonnestroom,mln kWh,.,.,.,.
1516,Gemeenten; niet in te delen,Installaties,aantal,426,509,520,533
1517,Gemeenten; niet in te delen,Opgesteld vermogen van zonnepanelen,kWp,14389,17208,28679,175854
1518,Gemeenten; niet in te delen,Opgesteld vermogen omvormers,kW,NaN,14768,23689,140124
1519,Gemeenten; niet in te delen,Productie van zonnestroom,mln kWh,12,16,25,28


Next we will drop the unnamed column, as this column is not necessary.

In [5]:
df = df.drop(columns=['Unnamed: 2'])
df.head()

,Regio's,Onderwerp,2021,2022,2023 **,2024 eerste helft*
0,Nederland,Installaties,1730285,2298859,2875487,3023653
1,Nederland,Opgesteld vermogen van zonnepanelen,14822880,19536096,24301725,26065113
2,Nederland,Opgesteld vermogen omvormers,NaN,17451365,21394768,22814464
3,Nederland,Productie van zonnestroom,11304,16659,19580,10984
4,Noord-Nederland (LD),Installaties,246365,309909,379524,398051


We will also rename the year columns to only contain the year and no other characters.

In [6]:
df = df.rename(columns={'2023 **': '2023', '2024 eerste helft*': '2024'})
df.head()

,Regio's,Onderwerp,2021,2022,2023,2024
0,Nederland,Installaties,1730285,2298859,2875487,3023653
1,Nederland,Opgesteld vermogen van zonnepanelen,14822880,19536096,24301725,26065113
2,Nederland,Opgesteld vermogen omvormers,NaN,17451365,21394768,22814464
3,Nederland,Productie van zonnestroom,11304,16659,19580,10984
4,Noord-Nederland (LD),Installaties,246365,309909,379524,398051


Finally we will change the format of the dataframe from a wide format to a long format, by switching the `Onderwerp` values with the years.

In [7]:
df_long = pd.melt(df, id_vars=["Regio's", 'Onderwerp'], var_name='Year', value_name='Value')

df_long['Value'] = df_long['Value'].replace('.', np.nan)
df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')

In [8]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Regio's    6080 non-null   object 
 1   Onderwerp  6080 non-null   object 
 2   Year       6080 non-null   object 
 3   Value      4091 non-null   float64
dtypes: float64(1), object(3)
memory usage: 190.1+ KB


Check for duplicates before we pivot the dataframe.

In [9]:
duplicates = df_long[df_long.duplicated(subset=['Year', 'Onderwerp', "Regio's"], keep=False)]
duplicates

,Regio's,Onderwerp,Year,Value


There appear to be no duplicate rows.

In [10]:
df_pivot = df_long.pivot(index=['Year', "Regio's"], columns='Onderwerp', values='Value').reset_index()
df_pivot.columns.name = None

df_pivot

,Year,Regio's,Installaties,Opgesteld vermogen omvormers,Opgesteld vermogen van zonnepanelen,Productie van zonnestroom
0,2021,'s-Gravenhage (gemeente),17989.0,NaN,77435.0,NaN
1,2021,'s-Hertogenbosch,13883.0,NaN,103745.0,NaN
2,2021,Aa en Hunze,4392.0,NaN,33382.0,NaN
3,2021,Aalsmeer,3093.0,NaN,34473.0,NaN
4,2021,Aalten,3346.0,NaN,33100.0,NaN
...,...,...,...,...,...,...
1515,2024,Zundert,4548.0,39621.0,45963.0,NaN
1516,2024,Zutphen,8238.0,55888.0,62270.0,NaN
1517,2024,Zwartewaterland,4620.0,37664.0,41483.0,NaN
1518,2024,Zwijndrecht,6638.0,100421.0,105131.0,NaN


Finally we will export the cleaned dataframe, so we can use it for further analysis.

In [12]:
file_path = '../data/Other/cleaned_solar_panel_data.csv'
df_pivot.to_csv(file_path, index=False)